In [17]:
!git clone https://github.com/ytsrt66589/pyneuralfx.git

Cloning into 'pyneuralfx'...
remote: Enumerating objects: 321, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 321 (delta 22), reused 39 (delta 13), pack-reused 263 (from 1)
Receiving objects: 100% (321/321), 95.88 MiB | 2.97 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [5]:
%ls

configs/       example_wavs/   main_full.py      __pycache__/  utils.py
data/          exp/            main_snapshot.py  pyneuralfx/   vis_examples/
dataset.py     grid_search.py  preprocess/       saver.py      visualization.py
evaluation.py  loss_analysis/  pre_trained/      solver.py


In [ ]:
!pip install -r requirements.txt

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [4]:
%cd pyneuralfx/frame_work

/home/fu/Documents/vocaloid_modeling/vocaloid_pyneuralfx/pyneuralfx/frame_work


In [3]:
%pwd

'/home/fu/Documents/vocaloid_modeling/vocaloid_pyneuralfx'

# Initialization

In [2]:
!python3 preprocess/preproc_snapshot.py

> ================= Start Preprocessing =================  <
> ================= Finish Preprocessing =================  <


In [4]:
import torch
print(torch.cuda.is_available())

True


# Grid Search
(Modify parameter space in `grid_search.py`)

In [23]:
!python3 grid_search.py

Grid Search Progress:   0%|                               | 0/4 [00:00<?, ?it/s]=======================
train_x_path: ../../../data/audio/sv_teto_0_0_0_0_MixDown.wav
train_y_path: ../../../data/audio/sv_teto_0_1_1_1_MixDown.wav
> causal: False
> dilation_growth: 2
> n_channel: 32
> epochs: 20
> lr: 0.0005
> batch_size: 40
> kernel_size: 7
> n_blocks: 10
> ========= Hybrid ============== <
> [Loss] --- Hybrid Trans Loss ---
> [Loss] --- Temporal L1 Loss ---
> [Loss] --- Multi-resolution STFT Loss ---
> ========= Hybrid ============== <
> [Loss] --- Hybrid Trans Loss ---
> [Loss] --- Temporal L1 Loss ---
> [Loss] --- Multi-resolution STFT Loss ---
EXP DIR:  exp/vocaloid/snapshot_tcn
 >>>>> training
> train dataset ready ...........
> valid dataset ready ...........
 [!] saver created!
 > params amount: 74,945 | trainable: 74,945 |  bs: 40  
============ start training ============
epoch: 0/20 (  0/162) | exp/vocaloid/snapshot_tcn | t: 0.68 | loss: 5.869799 | time: 0:00:00.7 | counter: 0


# Snapshot Training for Best Parameters
(Please modify the model configuration in `./config/cnn/tcn/snapshot_tcn.yml` based on the best parameters given above)

In [8]:
!python3 main_snapshot.py

 > config: ./configs/cnn/tcn/snapshot_tcn.yml
> ========= Hybrid ============== <
> [Loss] --- Hybrid Trans Loss ---
> [Loss] --- Temporal L1 Loss ---
> [Loss] --- Multi-resolution STFT Loss ---
> ========= Hybrid ============== <
> [Loss] --- Hybrid Trans Loss ---
> [Loss] --- Temporal L1 Loss ---
> [Loss] --- Multi-resolution STFT Loss ---
EXP DIR:  exp/vocaloid/snapshot_tcn
{'data': {'buffer_size': 2048, 'sampling_rate': 44100, 'inp_channels': 1, 'out_channels': 1, 'train_x_path': '../../../data/audio/sv_teto_0_0_0_0_MixDown.wav', 'train_y_path': '../../../data/audio/sv_teto_0_1_1_1_MixDown.wav', 'valid_x_path': '../../../data/audio/sv_teto_0_0_0_0_MixDown.wav', 'valid_y_path': '../../../data/audio/sv_teto_0_1_1_1_MixDown.wav', 'test_x_path': '../../../data/audio/sv_teto_0_0_0_0_MixDown.wav', 'test_y_path': '../../../data/audio/sv_teto_0_1_1_1_MixDown.wav'}, 'model': {'arch': 'snapshot-tcn', 'n_blocks': 9, 'kernel_size': 12, 'dilation_growth': 2, 'n_channels': 32, 'causal': True, 'p

In [9]:
!python3 evaluation.py

> [Loss] --- Multi-resolution STFT Loss ---
> [Loss] --- ESR Loss ---
> [Metrics] --- Transient evaluation based on STN separation---
> [Loss] --- ESR Loss ---
> [Metrics] --- LUFS ---
> [Metrics] --- Crest Factor (Dynamics range) [dB] ---
> [Metrics] --- Rms energy (Dynamics range) [dB] ---
> [Metrics] --- Spectral Centroid ---
> exp name:  vocaloid/snapshot_tcn
> total audio clips:  1
  0%|                                                     | 0/1 [00:00<?, ?it/s]/home/fu/Documents/vocaloid_modeling/.venv/lib/python3.9/site-packages/torch/functional.py:709: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /pytorch/aten/src/ATen/native/SpectralOps.cpp:873.)
  return _VF.stft(  # type: ignore[attr-defined]
/home/fu/Documents/vocalo

In [11]:
import librosa
import IPython.display as ipd
input, sr = librosa.load('./exp/vocaloid/snapshot_tcn/valid_gen/inp/output0.wav', sr=None, mono=True)
target_y, sr_x = librosa.load('./exp/vocaloid/snapshot_tcn/valid_gen/anno/output0.wav', sr=None, mono=True)
pred_y, sr_y = librosa.load('./exp/vocaloid/snapshot_tcn/valid_gen/pred/output0.wav', sr=None, mono=True)
assert sr_x == sr_y
print('Input')
ipd.display(ipd.Audio(data=input[:sr*5], rate=sr, normalize=False))
print('Target')
ipd.display(ipd.Audio(data=target_y[:sr_x*5], rate=sr_x, normalize=False))
print(f'Predict')
ipd.display(ipd.Audio(data=pred_y[:sr_y*5], rate=sr_y, normalize=False))

Input


Target


Predict
